In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier

from rubicon import Rubicon
from rubicon.sklearn import RubiconPipeline

In [ ]:
categories = ["alt.atheism", "talk.religion.misc"]
data = fetch_20newsgroups(subset='train', categories=categories)

In [ ]:
rubicon = Rubicon(persistence="memory")
project = rubicon.get_or_create_project("Rubicon Pipeline Example")

pipeline = RubiconPipeline(
    [
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', SGDClassifier()),
    ],
    project,
)

pipeline.fit(data.data, data.target)

In [ ]:
pipeline["clf"].__dict__

In [1]:
from rubicon.sklearn import get_logger

In [2]:
get_logger("there aren't any yet")

rubicon.sklearn.base_logger.BaseEstimatorLogger